In [1]:
# ! pip install pybaseball

In [2]:
# https://github.com/jldbc/pybaseball
import pybaseball as pyb
import pandas as pd


In [3]:
# mk_pid = pyb.playerid_lookup('kelly', 'michael')
# mk_pid

In [4]:
# mk_pid[['key_mlbam']]
mk_stats = pyb.statcast_pitcher('2022-01-01', '2024-12-31', 547184)

Gathering Player Data


In [5]:
# List of columns to average
columns_to_avg = [
    'release_speed', 'release_pos_x', 'release_pos_z', 'pfx_x','pfx_z', 'plate_x','plate_z',
    'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 
    'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 
    'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension'
]

In [6]:
# pd.set_option('display.max_columns', None)
mk_stats['unique_at_bat'] = mk_stats['game_pk'].astype(str) + '_' + mk_stats['at_bat_number'].astype(str)

mk_stats.columns
mk_stats.sample(5)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,unique_at_bat
635,ST,2023-09-19,83.4,-1.30,5.98,"Kelly, Michael",664728,547184,NaN,ball,...,4,0,Infield shade,Standard,46.0,0.000,0.012,NaN,NaN,716517_36
326,FF,2024-04-15,94.2,-1.23,6.20,"Kelly, Michael",663457,547184,NaN,foul,...,3,0,Standard,Standard,204.0,0.000,-0.057,37.903367,4.28342,745677_51
483,FF,2024-03-25,94.8,-0.99,6.34,"Kelly, Michael",682617,547184,double,hit_into_play,...,2,0,Standard,Standard,204.0,-0.047,NaN,NaN,NaN,747807_31
972,FF,2022-04-03,94.8,-1.28,6.28,"Kelly, Michael",686531,547184,strikeout,swinging_strike,...,7,8,Standard,Standard,200.0,0.091,NaN,NaN,NaN,706865_73
316,FF,2024-04-15,94.7,-1.37,6.04,"Kelly, Michael",669357,547184,NaN,foul,...,3,0,Infield shade,Standard,203.0,0.000,0.000,57.642203,5.30702,745677_57


In [8]:
# display(mk_stats.sample(5))
# print(mk_stats.columns)
# Group by 'pitch_type' and 'game_year', then calculate the average of specified columns
grouped_stats = mk_stats.groupby(['pitcher','pitch_type','pitch_name', 'game_year']).agg({
    'description': 'size',  # This will count the number of rows in each group
    'game_pk': 'nunique',
    'unique_at_bat':'nunique',
    **{col: 'mean' for col in columns_to_avg}
}).reset_index()

# Rename the count column
grouped_stats = grouped_stats.rename(columns={'description': 'event_count', 'game_pk':'num_games','unique_at_bat':'num_pas'})

# Calculate the total pitches for each year
year_totals = grouped_stats.groupby('game_year')['event_count'].sum().reset_index(name='year_total')

# Merge the year totals back into the main dataframe
grouped_stats = pd.merge(grouped_stats, year_totals, on='game_year')

# Calculate the percentage of total for each year
grouped_stats['pct_of_year'] = grouped_stats['event_count'] / grouped_stats['year_total'] * 100

# Sort the dataframe
grouped_stats = grouped_stats.sort_values(['game_year', 'event_count'], ascending = [False, False])
# Display the result

grouped_stats


,pitcher,pitch_type,pitch_name,game_year,event_count,num_games,num_pas,release_speed,release_pos_x,release_pos_z,...,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,year_total,pct_of_year
11,547184,ST,Sweeper,2024,268,31,114,83.251119,-1.349478,6.028694,...,3.392164,1.603134,155.792453,79.105714,15.257143,82.190672,2746.507463,5.855224,506,52.964427
8,547184,FF,4-Seam Fastball,2024,166,28,91,95.086145,-1.259337,6.157771,...,3.401265,1.619036,159.875000,81.187500,17.895833,94.594578,2344.963855,5.941566,506,32.806324
10,547184,SL,Slider,2024,68,24,56,88.361765,-1.366765,6.114559,...,3.422500,1.620735,157.458333,86.300000,17.652174,87.723529,2685.323529,5.854412,506,13.438735
9,547184,SI,Sinker,2024,4,4,4,92.100000,-1.275000,6.087500,...,3.410000,1.625000,14.500000,79.250000,-23.500000,91.300000,1990.000000,5.875000,506,0.790514
7,547184,ST,Sweeper,2023,145,14,58,83.306897,-1.158000,5.962345,...,3.336000,1.574552,156.777778,79.573333,21.711111,82.714483,2735.413793,6.042759,272,53.308824
4,547184,FF,4-Seam Fastball,2023,62,13,38,94.995161,-1.023226,6.066290,...,3.338871,1.578548,164.000000,89.817647,17.764706,95.159677,2234.919355,6.188710,272,22.794118
6,547184,SL,Slider,2023,45,12,26,88.226667,-1.147111,6.004000,...,3.466222,1.660000,176.875000,77.950000,17.687500,88.148889,2619.088889,6.097778,272,16.544118
5,547184,SI,Sinker,2023,20,8,13,94.565000,-1.110500,5.933000,...,3.391000,1.615500,105.500000,84.050000,25.250000,94.500000,2052.450000,6.195000,272,7.352941
3,547184,ST,Sweeper,2022,106,11,38,82.865094,-1.222830,6.095189,...,3.419340,1.591038,157.593750,78.996970,26.030303,82.192632,2576.875000,6.020833,204,51.960784
1,547184,FF,4-Seam Fastball,2022,69,11,39,94.218841,-1.080725,6.217971,...,3.386377,1.568406,188.925926,85.803704,26.037037,94.125000,2243.642857,6.214286,204,33.823529
